In [ ]:
#installation pyspark dans notebook
! pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [4]:
#import des librairies
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [17]:
spark

# Decouverte du jeu de donnée

In [22]:
#import jeu de donnée
data_tree = spark.read.format("libSvm").load("/home/menan/data_science/pyspark/sample_libsvm_data.txt")

22/07/15 16:03:52 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [23]:
#decouverte du jeu de donnée
data_tree.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
#les colonnes du jeu de donnée
data_tree.columns

['label', 'features']

In [26]:
#nombre de ligne du jeu de donnée
data_tree.count()

100

In [24]:
#5 premières ligne du jeu de donnée
data_tree.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



# Devéloppement modèle et prédiction

In [28]:
#pour information : le jeu de donnée est déjà prétraiter.

In [29]:
#separation apprentissage-test
(trainSet, testSet) = data_tree.randomSplit([0.7, 0.3])

In [31]:
#vérification des proportions dans chaque subset
trainSet.count(), testSet.count()

(68, 32)

In [32]:
#definition du modèle
random_forest = RandomForestClassifier(
    labelCol = "label", featuresCol="features", numTrees=20) 

In [35]:
#entrainememnt du modèle
random_rf = random_forest.fit(trainSet)

In [36]:
#prediction sur les données de test
prediction = random_rf.transform(testSet)

In [37]:
#schema de la variable prediction
prediction.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [40]:
#affichage de la proba de prédiction, de la prédiction et les features
#des 5 premiers individus du jeu de test
prediction.select("label","prediction", "probability", 'features').show(5)

+-----+----------+-----------+--------------------+
|label|prediction|probability|            features|
+-----+----------+-----------+--------------------+
|  0.0|       0.0|[0.95,0.05]|(692,[95,96,97,12...|
|  0.0|       0.0|[0.85,0.15]|(692,[123,124,125...|
|  0.0|       0.0|  [1.0,0.0]|(692,[124,125,126...|
|  0.0|       0.0|  [1.0,0.0]|(692,[124,125,126...|
|  0.0|       0.0|  [0.7,0.3]|(692,[124,125,126...|
+-----+----------+-----------+--------------------+
only showing top 5 rows



In [54]:
#evaluation du modèle
evaluator_acc = MulticlassClassificationEvaluator(
    labelCol = 'label',
    predictionCol= 'prediction',
    metricName= 'accuracy')


accuracy = evaluator_acc.evaluate(prediction)

    
#print("Test Error = %g" % (1.0 - accuracy))

print("accuracy du modèle :%g" %(1.0-accuracy))

accuracy du modèle :0.03125


# Conclusion

In [ ]:
#le modèle à une performance de plus de 95%.